In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from torchinfo import summary

In [33]:
# Hyper-Parameters
epochs = 5
batch_size = 20
learning_rate = 0.001

class Inception_Mod(nn.Module):
    def __init__(self, prev_layer):
        super(Inception_Mod, self).__init__()  
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=prev_layer, out_channels=6, kernel_size=1)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=prev_layer, out_channels=6, kernel_size=1),
            nn.Conv2d(in_channels=6, out_channels= 6, kernel_size=3, stride=1, padding=1)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=prev_layer, out_channels=6, kernel_size=1),
            nn.Conv2d(in_channels=6, out_channels=6, kernel_size=5, padding = 2, stride=1)
        )
        self.block4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1),
            nn.Conv2d(in_channels=prev_layer, out_channels=6,kernel_size=1, padding=1)
        )
        
    def forward(self, x):
        first_block = self.block1(x)
        second_block = self.block2(x)
        third_block = self.block3(x)
        fourth_block = self.block4(x)

        outputs_concat = torch.cat([first_block, second_block, third_block, fourth_block], dim =1)
        return outputs_concat

In [36]:
hep_data = torch.randn(1,1, 256, 256)

# model = Inception_Mod(6)
# out = model()

a = create_inceptionMod(hep_data)
print(a.shape)

torch.Size([1, 24, 256, 256])


In [16]:
def create_inceptionMod(prev_layer):
        inception_mod = Inception_Mod(prev_layer.shape[1])
        output = inception_mod(prev_layer)
        return output

def createBranches(data):
        x_view = torch.unsqueeze(data[0], dim = 0)
        y_view = torch.unsqueeze(data[1], dim = 0)
        return x_view, y_view
def mergeXY_views(x_view, y_view):
        return torch.cat([x_view, y_view], dim=0)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# image size = 1 x256 x 256
random_tensor = torch.randn(1, 1, 256, 256)

class CVN_Net(nn.Module):
    def __init__(self):
        super(CVN_Net,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6,  kernel_size=7, stride=2)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride = 2)        
        self.lrn_norm = nn.LocalResponseNorm(size=5,  alpha=0.0001, beta=0.75)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels= 12, kernel_size=1)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=6, kernel_size=3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool(x)
        x = self.lrn_norm(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.lrn_norm(x)
        x = self.max_pool(x)

        inception_mod_output1 = create_inceptionMod(x)
        inception_mod_output2 = create_inceptionMod(inception_mod_output1)
        x = self.max_pool(inception_mod_output2)
        inception_mod_output3 = create_inceptionMod(x)
        return inception_mod_output3


model = CVN_Net().to(device=device)
output = model(random_tensor)

[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


In [8]:
summary(model, input_size=(1, 1,256,256))

Layer (type:depth-idx)                   Output Shape              Param #
CVN_Net                                  [64, 6, 14, 14]           --
├─Conv2d: 1-1                            [1, 6, 125, 125]          300
├─MaxPool2d: 1-2                         [1, 6, 62, 62]            --
├─LocalResponseNorm: 1-3                 [1, 6, 62, 62]            --
├─Conv2d: 1-4                            [1, 12, 62, 62]           84
├─Conv2d: 1-5                            [1, 6, 60, 60]            654
├─LocalResponseNorm: 1-6                 [1, 6, 60, 60]            --
├─MaxPool2d: 1-7                         [1, 6, 29, 29]            --
├─MaxPool2d: 1-8                         [16, 6, 14, 14]           --
Total params: 1,038
Trainable params: 1,038
Non-trainable params: 0
Total mult-adds (M): 7.36
Input size (MB): 0.26
Forward/backward pass size (MB): 1.29
Params size (MB): 0.00
Estimated Total Size (MB): 1.56

## Testing entire Architecture

In [13]:
hep_data = torch.randn(2,1, 256, 256)
x_data, y_data = createBranches(hep_data)


x_model = CVN_Net().to(device=device)
y_model = CVN_Net().to(device=device)
x_output = x_model(x_data)
y_output = y_model(y_data)
print(f'Shape of X_View Data: {x_output.shape}')
print(f'Shape of Y_View Data: {y_output.shape}')
concatXY_views = torch.concat([x_output, y_output], dim = 0)
print(concatXY_views.shape)
final_inception_output = create_inceptionMod(concatXY_views)
# avg_pooling = nn.AvgPool2d(kernel_size=(6,5))
# output = avg_pooling(final_inception_output)
# softmax = nn.LogSoftmax(dim = 0)
# output = softmax(output)

Shape of X_View Data: torch.Size([64, 6, 14, 14])
Shape of Y_View Data: torch.Size([64, 6, 14, 14])
torch.Size([128, 6, 14, 14])


# Combining Torch Modules

In [37]:
class Inception_Mod(nn.Module):
    def __init__(self, prev_layer):
        super(Inception_Mod, self).__init__()  
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=prev_layer, out_channels=6, kernel_size=1)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=prev_layer, out_channels=6, kernel_size=1),
            nn.Conv2d(in_channels=6, out_channels= 6, kernel_size=3, stride=1, padding=1)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=prev_layer, out_channels=6, kernel_size=1),
            nn.Conv2d(in_channels=6, out_channels=6, kernel_size=5, padding = 2, stride=1)
        )
        self.block4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1),
            nn.Conv2d(in_channels=prev_layer, out_channels=6,kernel_size=1, padding=1)
        )
        
    def forward(self, x):
        first_block = self.block1(x)
        second_block = self.block2(x)
        third_block = self.block3(x)
        fourth_block = self.block4(x)

        outputs_concat = torch.cat([first_block, second_block, third_block, fourth_block], dim  = 0)
        return outputs_concat
    
class x_model(nn.Module):
    def __init__(self):
        super(x_model,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6,  kernel_size=7, stride=2)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride = 2)        
        self.lrn_norm = nn.LocalResponseNorm(size=5,  alpha=0.0001, beta=0.75)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels= 12, kernel_size=1)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=6, kernel_size=3)
        

    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.lrn_norm(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.max_pool(self.lrn_norm(x))
        x = Inception_Mod(x.shape[0])
        x = Inception_Mod(x.shape[0])


        return x
class y_model(nn.Module):
    def __init__(self):
        super(y_model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6,  kernel_size=7, stride=2)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride = 2)        
        self.lrn_norm = nn.LocalResponseNorm(size=5,  alpha=0.0001, beta=0.75)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels= 12, kernel_size=1)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=6, kernel_size=3)
    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.lrn_norm(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.max_pool(self.lrn_norm(x))

        return x

class combined_model(nn.Module):
    def __init__(self):
        super(combined_model, self).__init__()
        self.model_x = x_model()
        self.model_y = y_model()
        self.combined_inception = Inception_Mod()
        self.avg_pooling  = nn.AvgPool2d(kernel_size=(6,5))

    def forward(self, x_data, y_data):
        x_data = self.model_x(x_data)
        y_data = self.model_y(y_data)

        x = torch.cat((x_data, y_data), dim = 1)
        x = self.combined_inception(x)
        # Don't use softmax, the torch Cross Entropy function
        # already uses softmax on the logits returned.
        return x

# Adjustments with Paper Parameters

In [38]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)

    def forward(self, x):
        return F.relu(self.conv(x))

In [39]:
class Inception_Block(nn.Module):
    def __init__(
            self,
            in_channels,
            output_1x1,
            input_3x3,
            output_3x3,
            input_5x5,
            output_5x5,
            output_pool,
    ):
      super(Inception_Block, self).__init__()
      self.branch1 = ConvBlock(in_channels, output_1x1, kernel_size = 1)
      self.branch2 = nn.Sequential(
            ConvBlock(in_channels, input_3x3, kernel_size = 1, padding = 0),
            ConvBlock(input_3x3, output_3x3, kernel_size = 3, padding = 1)
      )
      self.branch3 = nn.Sequential(
            ConvBlock(in_channels, input_5x5, kernel_size = 1),
            ConvBlock(input_5x5, output_5x5, kernel_size = 5, padding = 2),
      )
      self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1),
            ConvBlock(in_channels, output_pool, kernel_size = 1, padding = 1)
      )

      def forward(self, x):
          first_block = self.branch1(x)
          second_block = self.branch2(x)
          third_block = self.branch3(x)
          fourth_block = self.branch4(x)
          output_concat = torch.cat([first_block, second_block, third_block, fourth_block], dim  = 1)

          return output_concat
         